# Ulepszanie wydajności klasyfikatorów

Wydajność modeli można zwiększać poprzez:

- dostrajanie parametrów pojedynczego modelu w celu jego ulepszenia. Do automatycznego dostrajania parametrów modelu w R można użyć pakietu `caret`. Pełna lista metod (algorytmów uczenia maszynowego) i ich dostosowywalnych parametrów znajduje się na [stronie internetowej](http://topepo.github.io/caret/available-models.html).

- łączenie ze sobą kilku słabych modeli w celu utworzenia lepszej jednostki. Jest to tak zwane uczenie zespołowe (ang. *ensemble learning*), któremu poświęcone są dzisiejsze ćwiczenia.

## Metody zespołowe

Podstawowe założenie: zaangażowanie zespołu różnych ekspertów (modeli) we wspólne rozwiązywanie problemu podniesie wydajność uczenia.

### Bagging

Jedna z najpopularniejszych technik uczenia zespołowego. Nazwa pochodzi od *bootstrap aggregating* - technika bagging wykorzystuje próbkowanie metodą bootstrap w funkcji alokacji do generowania danych przypisywanych do każdego z modeli w zespole. Zespoły w agregacji bootstrap zazwyczaj składają się z jednorodnych klasyfikatorów, które są trenowane równolegle i niezależnie od siebie.
Wśród metod bagging jedną z najpopularniejszych są *lasy losowe* (ang. *random forest*), składające się z wielu klasyfikatorów drzew decyzyjnych. Funkcja alokacji lasów łączy próbkowanie metodą bootstrap z losowym wybieraniem cech do generowania danych przydzielanych każdemu z klasyfikatorów w zespole. Lasy losowe nadają się do obsługiwania bardzo szerokich zbiorów danych.

### Boosting

Zespoły Wzmacniające (ang. *boosting*) są również jednorodnymi zbiorami podstawowych modeli. W odróżnieniu od agregacji bootstrap, we wzmacnianiu podstawowe modele są trenowane sekwencyjnie. Każdy kolejny model w sekwencji próbuje osiągnąć większą wydajność od poprzedniego modelu, ucząc się na błędach poprzednika. Następnie przeprowadzane jest wtórne próbkowanie danych treningowych tak, że przykładom dla których były nieprawidłowe predykcje, otrzymują większą wagę.

Proces wtórnego próbkowania, trenowania, oceniania i punktowania jest powtarzany do każdego modelu w sekwencji, dopóki wszystkie modele nie zostaną wytrenowane.

### Stacking

Stacking, czyli kontaminacja modeli, różni się od poprzednich dwóch tym, że zawiera tu zwykle różnorodne modele podstawowe. Przykładowo może składać się z modelu kNN, modelu regresji logistycznej oraz naiwnego modelu Bayesa. Kontaminacja modeli jest podobna do agregacji bootstrap, tyle że funkcja kombinacji jest tutaj niedeterministyczna (nie przebiega zgodnie z predefiniowanym wzorcem czy zbiorem reguł).

Zazponaj się z materiałami zamieszczonymi na stronie [RPubs](https://rpubs.com/mmazurek/346331), sekcje:

- Klasyfikatory złożone

- Agregacja bootstrapowa (bagging - bootstrap aggregation)

- Lasy losowe

- Boosting
  - Adaboost - adaptive boosting

Następnie wykonaj poniższe zadania.

## Zadania

1.	Pobierz i wczytaj w RStudio plik z danymi `Movie_classification.csv`. Celem zadania jest przewidzenie, czy dany film otrzyma Oskara czy też nie (zmienna dychotomiczna `Start_Tech_Oskar`).

In [ ]:
#install.packages("caTools")
#install.packages("caret")
#install.packages("randomForest")
#install.packages("tidyverse")
#install.packages("dplyr")
#install.packages("adabag")


library(tidyverse)
library(caTools)
library(caret)
library(randomForest)
library(adabag)
library(dplyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bitops’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘XML’, ‘rgl’, ‘rlist’, ‘gtools’, ‘doParallel’, ‘ConsRank’


── Attaching core tidyverse packages ────────────

In [ ]:
dane <- read.csv("Movie_classification.csv")

glimpse(dane)

Rows: 506
Columns: 19
$ Marketing.expense   <dbl> 20.1264, 20.5462, 20.5458, 20.6474, 21.3810, 20.59…
$ Production.expense  <dbl> 59.62, 69.14, 69.14, 59.36, 59.36, 59.36, 70.74, 7…
$ Multiplex.coverage  <dbl> 0.462, 0.531, 0.531, 0.542, 0.542, 0.542, 0.476, 0…
$ Budget              <dbl> 36524.12, 35668.65, 39912.68, 38873.89, 39701.58, …
$ Movie_length        <dbl> 138.7, 152.4, 134.6, 119.3, 127.7, 132.2, 140.1, 1…
$ Lead_.Actor_Rating  <dbl> 7.825, 7.505, 7.485, 6.895, 6.920, 6.890, 7.065, 6…
$ Lead_Actress_rating <dbl> 8.095, 7.650, 7.570, 7.035, 7.070, 7.100, 7.265, 7…
$ Director_rating     <dbl> 7.910, 7.440, 7.495, 6.920, 6.815, 6.885, 7.150, 6…
$ Producer_rating     <dbl> 7.995, 7.470, 7.515, 7.020, 7.070, 7.005, 7.400, 7…
$ Critic_rating       <dbl> 7.94, 7.44, 7.44, 8.26, 8.26, 7.26, 8.96, 7.96, 7.…
$ Trailer_views       <int> 527367, 494055, 547051, 516279, 531448, 498425, 45…
$ X3D_available       <chr> "YES", "NO", "NO", "YES", "NO", "YES", "YES", "NO"…
$ Time_taken      

2.	Przyglądnij się danym i przygotuj je do dalszej analizy:

 - braki danych zastąp wartością średnią dla danego atrybutu,
 - za pomocą funkcji `sample.split` (z pakietu `caTools`) oraz `subset` podziel dane na zbiór treningowy i testowy.

In [ ]:
# Zamieniamy NA na wartości średnie
dane_cleaned <- dane %>%
  mutate(across(where(is.numeric), ~ ifelse(is.na(.), mean(., na.rm = TRUE), .)))

In [ ]:
# Zamieniamy typ zmiennej na factor
dane_cleaned$Start_Tech_Oscar <- as.factor(dane_cleaned$Start_Tech_Oscar)

In [ ]:
# Podzielenie danych na train i test
set.seed(123)
data_split <- sample.split(dane_cleaned$Start_Tech_Oscar, SplitRatio = 0.7)
train <- subset(dane_cleaned, data_split == TRUE)
test <- subset(dane_cleaned, data_split == FALSE)

3. Dokonaj klasyfikacji za pomocą trzech algorytmów złożonych: bagging, random forest oraz boosting (zbuduj modele na zbiorze treningowym).

# Bagging

In [ ]:
bagging_model <- bagging(Start_Tech_Oscar ~ ., data = train, mfinal = 100)
bagging_pred_train <- predict(bagging_model, train)
bagging_pred_test <- predict(bagging_model, test)

# Random Forest

In [ ]:
rf_model <- randomForest(Start_Tech_Oscar ~ ., data = train,
                        ntree = 500,  # liczba drzew
                        mtry = sqrt(ncol(train)-1))  # liczba zmiennych na kazdym drzewie
rf_pred_train <- predict(rf_model, train)
rf_pred_test <- predict(rf_model, test)

# Boosting

In [ ]:
# AdaBoost
boosting_model <- boosting(Start_Tech_Oscar ~ ., data = train, mfinal = 100)
boosting_pred_train <- predict(boosting_model, train)
boosting_pred_test <- predict(boosting_model, test)

4. Sprawdź jak dobrze utworzone modele radzą sobie z danymi testowymi (użyj funkcji `confusionMatrix` z pakietu `caret`). Jaki procent danych został poprawnie zaklasyfikowany? Który model okazał się w tym przypadku najlepszy?

In [ ]:
library(caret)

evaluate_model <- function(actual, predicted, model_name) {
  conf_matrix <- confusionMatrix(predicted, actual)
  cat("\nResults for", model_name, ":\n")
  cat("Accuracy:", round(conf_matrix$overall["Accuracy"] * 100, 2), "%\n")
  print(conf_matrix$table)
  cat("\n")
}

# Ocena modelu Baggingu

In [ ]:
evaluate_model(test$Start_Tech_Oscar, as.factor(bagging_pred_test$class), "Bagging")


Results for Bagging :
Accuracy: 61.84 %
          Reference
Prediction  0  1
         0 41 30
         1 28 53



# Random Forest

In [ ]:
evaluate_model(test$Start_Tech_Oscar, rf_pred_test, "Random Forest")


Results for Random Forest :
Accuracy: 65.13 %
          Reference
Prediction  0  1
         0 44 28
         1 25 55



# Boosting

In [ ]:
evaluate_model(test$Start_Tech_Oscar, as.factor(boosting_pred_test$class), "Boosting")


Results for Boosting :
Accuracy: 64.47 %
          Reference
Prediction  0  1
         0 41 26
         1 28 57

